In [5]:
import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from shapely.geometry import Point
from geopandas import GeoDataFrame
import os

import seaborn as sns
pd.set_option("display.max_colwidth", 0)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.options.display.float_format = '{:.5f}'.format

import warnings
warnings.filterwarnings("ignore")

In [2]:
%%time
df_localness = pd.read_csv('./data/food_localness_oct.csv',header=0) #,nrows=10
# df_localness

CPU times: user 13.7 s, sys: 2.22 s, total: 15.9 s
Wall time: 46 s


In [7]:
df_localness['customer_loc_id'] = df_localness['customer_lon'].astype(str)+df_localness['customer_lat'].astype(str)

## Create weekend binary column

In [3]:
%%time
# Create one column to determine if it is weekend
df_localness['order_date'] = pd.to_datetime(df_localness['order_date']) 

# Define a function to check if a date falls on a weekend
def is_weekend(date):
    return (date.dayofweek == 5) or (date.dayofweek == 6)

# Apply the function to the 'Date' column
df_localness['isweekend'] = df_localness['order_date'].apply(lambda x: is_weekend(x))

CPU times: user 6.75 s, sys: 156 ms, total: 6.9 s
Wall time: 17.6 s


## Merge with mean price per item (proxy for expensiveness of restaurants)

In [5]:
df_priceper_item = pd.read_csv('./data/mean_price_per_item.csv')
df_localness = df_localness.merge(df_priceper_item, how='left',on='vendor_id')
print(df_localness.shape[0])

df_localness = df_localness[df_localness['item_price']!=0]
print(df_localness.shape[0])

2828913


## merge entropy and restaurant count data

In [8]:
%%time
distance_list = [0.5, 1, 2, 3, 4, 5] #unit:km

# left join various entropy indices and number of resturants
for distance_threshold in distance_list:
    entropy_col_name = 'entropy_'+str(distance_threshold)+'km'
    poi_col_name = 'poi_count_'+str(distance_threshold)+'km'
    entropy_file_path = './data/entropy_'+str(distance_threshold)+'km_customerlocations_oct.csv'
    poicount_file_path = './data/poi_count_'+str(distance_threshold)+'km_customerlocations_oct.csv'
    
    entropy_bycustomerloc1 = pd.read_csv(entropy_file_path,header=0)
    df_localness = df_localness.merge(entropy_bycustomerloc1, how='left',on='customer_loc_id')
    df_localness[entropy_col_name] = df_localness[entropy_col_name].fillna(0)
    
    poi_bycustomerloc1 = pd.read_csv(poicount_file_path,header=0)
    df_localness = df_localness.merge(poi_bycustomerloc1, how='left',on='customer_loc_id')
    df_localness[poi_col_name] = df_localness[poi_col_name].fillna(0)
    print(df_localness.shape[0])

2828913
2828913
2828913
2828913
2828913
2828913
CPU times: user 31.5 s, sys: 25.9 s, total: 57.3 s
Wall time: 2min


In [9]:
df_localness['poi_count_0.5_1km'] = df_localness['poi_count_1km'] - df_localness['poi_count_0.5km']
df_localness['poi_count_1_2km'] = df_localness['poi_count_2km'] - df_localness['poi_count_1km']
df_localness['poi_count_2_3km'] = df_localness['poi_count_3km'] - df_localness['poi_count_2km']
df_localness['poi_count_3_4km'] = df_localness['poi_count_4km'] - df_localness['poi_count_3km']
df_localness['poi_count_4_5km'] = df_localness['poi_count_5km'] - df_localness['poi_count_4km']

## Merge with rent data

In [20]:
unique_customerlocs_withrent = pd.read_csv('./data/estimated_rent_customerlocations_oct.csv')
print(unique_customerlocs_withrent.shape[0])
unique_customerlocs_withrent.head()

791511


,customer_loc_id,approxi_rent
0,55.21387725.155851,90.44289
1,55.213885121047525.1557155663109,90.65273
2,55.21368825.15563,91.13944
3,55.2138425.155878,90.46375
4,55.213881433010125.1557653365541,90.57859


In [24]:
df_localness = df_localness.merge(unique_customerlocs_withrent, how='left',on='customer_loc_id')
print(df_localness.shape[0])

2828913


In [25]:
df_localness.head(2)

,account_id,order_id,order_date,order_time,vendor_id,vendor_name,main_cuisine,vertical,delivery_fee_amount_lc,service_fee_amount_lc,basket_amount_lc,is_incentive_applied,incentive_amount_lc,pickup_id,customer_lon,customer_lat,vendor_lon,vendor_lat,order_time_hour,delivery_duration,total_duration,node_id,nearest_dist1,poi1,n_vertical,n_vendor_id,n_main_cuisine,n_vendor_name,n_vendor_lon,n_vendor_lat,vendor_index,pandana_distance_realdist,pandana_localness,speed,isweekend,item_price,customer_loc_id,entropy_0.5km,poi_count_0.5km,entropy_1km,poi_count_1km,entropy_2km,poi_count_2km,entropy_3km,poi_count_3km,entropy_4km,poi_count_4km,entropy_5km,poi_count_5km,poi_count_0.5_1km,poi_count_1_2km,poi_count_2_3km,poi_count_3_4km,poi_count_4_5km,approxi_rent
0,7051337,1305576462,2023-10-10,2023-10-10 15:30:22,50868,"SHAGHF, Jumeirah 3",coffee,food,0.00000,1.73000,57.75000,True,10.00000,249162013.00000,55.21388,25.15585,55.23119,25.18903,15,17,27,7584090309,0.65335,840,food,697762,coffee,"Awaken Cafe, Al Wasl",55.21303,25.15146,840,4.61410,0.14160,10.25355,False,30.06471,55.21387725.155851,0.00000,0.00000,0.47137,21.00000,0.65855,105.00000,0.66518,198.00000,0.69070,296.00000,0.70457,582.00000,21.00000,84.00000,93.00000,98.00000,286.00000,90.44289
1,7051337,1313167366,2023-10-16,2023-10-16 16:43:34,50868,"SHAGHF, Jumeirah 3",coffee,food,7.50000,0.90000,26.25000,True,10.00000,250948408.00000,55.21389,25.15572,55.23119,25.18903,16,13,31,7584090309,0.65335,840,food,697762,coffee,"Awaken Cafe, Al Wasl",55.21303,25.15146,840,4.61410,0.14160,8.93051,False,30.06471,55.213885121047525.1557155663109,0.00000,0.00000,0.47137,21.00000,0.65855,105.00000,0.66603,197.00000,0.69169,295.00000,0.70585,587.00000,21.00000,84.00000,92.00000,98.00000,292.00000,90.65273


## merge incentives data

In [32]:
df_incentives = pd.read_csv('./data/mit_base_orders_incentives_oct.csv')
df_incentives.head(2)

,order_id,compensation,discounted delivery,free delivery,gem,item discount,menu discount,voucher wallet
0,1293286909,0.00000,0.00000,0.00000,0.00000,0.00000,30.00000,0.00000
1,1293286912,0.00000,0.00000,0.00000,0.00000,0.00000,30.00000,0.00000


In [33]:
%%time
df_localness = df_localness.merge(df_incentives, on='order_id',how='left')
print(df_localness.shape[0])
df_localness.fillna(0,inplace=True)

2828913
2796753


In [34]:
# create one categorical var for incentive types
df_localness['incentives_type'] = 'none'
df_localness.loc[df_localness['compensation'] != 0,'incentives_type'] = 'compensation'
df_localness.loc[df_localness['discounted delivery'] != 0,'incentives_type'] = 'discounted delivery'
df_localness.loc[df_localness['free delivery'] != 0,'incentives_type'] = 'free delivery'
df_localness.loc[df_localness['gem'] != 0,'incentives_type'] = 'gem'
df_localness.loc[df_localness['item discount'] != 0,'incentives_type'] = 'item discount'
df_localness.loc[df_localness['menu discount'] != 0,'incentives_type'] = 'menu discount'
df_localness.loc[df_localness['voucher wallet'] != 0,'incentives_type'] = 'voucher wallet'

In [35]:
# create three dummy vars for three most important incentives
df_localness['incentive_delivery'] = 'False'
df_localness.loc[df_localness['discounted delivery'] != 0,'incentive_delivery'] = 'True'
df_localness.loc[df_localness['free delivery'] != 0,'incentive_delivery'] = 'True'

df_localness['incentive_menu_discount'] = 'False'
df_localness.loc[df_localness['menu discount'] != 0,'incentive_menu_discount'] = 'True'

df_localness['incentive_voucher_wallet'] = 'False'
df_localness.loc[df_localness['voucher wallet'] != 0,'incentive_voucher_wallet'] = 'True'

## Merge with rating data

In [36]:
df_ratings = pd.read_csv('./data/restaurant_ratings.csv')
df_ratings = df_ratings[['vendor_id','rating']]
print(df_ratings.shape[0])
print(df_ratings.rating.value_counts())
df_ratings.head()

9349
Very good    6275
Amazing      2239
Good         698 
OK           137 
Name: rating, dtype: int64


,vendor_id,rating
0,728391,Amazing
1,637519,Amazing
2,707758,Amazing
3,673790,Amazing
4,8440,Very good


In [37]:
# left join with the picked resturants
df_localness = df_localness.merge(df_ratings, how='left')
print(df_localness.shape[0])
df_localness['rating'] = df_localness['rating'].fillna('None')
print(df_localness.shape[0])

2796753
2796753


In [38]:
# left join with nearest restaurants
df_ratings = df_ratings.rename(columns={"vendor_id": "n_vendor_id", "rating":"n_rating"})
df_localness = df_localness.merge(df_ratings, how='left')
df_localness['n_rating'] = df_localness['n_rating'].fillna('None')
print(df_localness.shape[0])

2796753


## Individual order consistency indicator: entropy
Lower value of entropy means high consistency

In [ ]:
df_consistency = pd.read_csv('./data/order_consistency_oct.csv')
print(df_consistency.shape[0])
df_consistency.head(2)

In [45]:
df_localness = df_localness.merge(df_consistency, how='left')
print(df_localness['consistency'].isnull().values.any())
print(df_localness.shape[0])

False
2796753


## Total number of orders in one month for each individual
entropy of consistency itself may be biased so also incorporate total number of orders each person placed in Oct

In [42]:
df_totalorders = df_localness[['account_id','vendor_id']].groupby('account_id').count().reset_index()
df_totalorders.columns = ['account_id', 'total_orders']
# df_totalorders.head()
df_localness = df_localness.merge(df_totalorders, how='left')
print(df_localness['total_orders'].isnull().values.any())
print(df_localness.shape[0])

False
2796753


## Rename columns to avoid confusions

In [46]:
# exclude rows without rent data
print('no rent data points:',df_localness[df_localness.approxi_rent==0].shape[0])
df_localness_filter = df_localness[df_localness.approxi_rent!=0]
print(df_localness_filter.shape[0])

no rent data points: 29467
2767286


In [4]:
df_localness_filter.loc[df_localness_filter.rating == 'Amazing', 'rating'] = '1.Amazing'
df_localness_filter.loc[df_localness_filter.rating == 'Very good', 'rating'] = '2.Very good'
df_localness_filter.loc[df_localness_filter.rating == 'Good', 'rating'] = '3.Good'
df_localness_filter.loc[df_localness_filter.rating == 'OK', 'rating'] = '4.OK'
df_localness_filter.loc[df_localness_filter.rating == 'None', 'rating'] = '5.None'

df_localness_filter.loc[df_localness_filter.n_rating == 'Amazing', 'n_rating'] = '1.Amazing'
df_localness_filter.loc[df_localness_filter.n_rating == 'Very good', 'n_rating'] = '2.Very good'
df_localness_filter.loc[df_localness_filter.n_rating == 'Good', 'n_rating'] = '3.Good'
df_localness_filter.loc[df_localness_filter.n_rating == 'OK', 'n_rating'] = '4.OK'
df_localness_filter.loc[df_localness_filter.n_rating == 'None', 'n_rating'] = '5.None'

df_localness_filter.loc[df_localness_filter.isweekend == True, 'isweekend'] = 'Yes'
df_localness_filter.loc[df_localness_filter.isweekend == False, 'isweekend'] = 'No'

df_localness_filter.loc[df_localness_filter.incentive_delivery == True, 'incentive_delivery'] = 'Yes'
df_localness_filter.loc[df_localness_filter.incentive_delivery == False, 'incentive_delivery'] = 'No'

df_localness_filter.loc[df_localness_filter.incentive_menu_discount == True, 'incentive_menu_discount'] = 'Yes'
df_localness_filter.loc[df_localness_filter.incentive_menu_discount == False, 'incentive_menu_discount'] = 'No'

df_localness_filter.loc[df_localness_filter.incentive_voucher_wallet == True, 'incentive_voucher_wallet'] = 'Yes'
df_localness_filter.loc[df_localness_filter.incentive_voucher_wallet == False, 'incentive_voucher_wallet'] = 'No'

In [5]:
df_localness_filter = df_localness_filter.rename(columns={"item_price": "price_per_item", "approxi_rent":"estimated_rent",
  "poi_count_0.5km" : "restaurant_count_0.5km", "poi_count_0.5_1km": "restaurant_count_0.5_1km",
  "poi_count_1_2km": "restaurant_count_1_2km", "poi_count_2_3km" : "restaurant_count_2_3km", 
  "poi_count_3_4km" : "restaurant_count_3_4km", "poi_count_4_5km" : "restaurant_count_4_5km"
})

## Correct localness value

In [12]:
df_localness_filter.head(2)

,account_id,order_id,order_date,order_time,vendor_id,vendor_name,main_cuisine,vertical,delivery_fee_amount_lc,service_fee_amount_lc,basket_amount_lc,is_incentive_applied,incentive_amount_lc,pickup_id,customer_lon,customer_lat,vendor_lon,vendor_lat,order_time_hour,delivery_duration,total_duration,node_id,nearest_dist1,poi1,n_vertical,n_vendor_id,n_main_cuisine,n_vendor_name,n_vendor_lon,n_vendor_lat,vendor_index,pandana_distance_realdist,pandana_localness,speed,isweekend,price_per_item,customer_loc_id,entropy_0.5km,restaurant_count_0.5km,entropy_1km,poi_count_1km,entropy_2km,poi_count_2km,entropy_3km,poi_count_3km,entropy_4km,poi_count_4km,entropy_5km,poi_count_5km,restaurant_count_0.5_1km,restaurant_count_1_2km,restaurant_count_2_3km,restaurant_count_3_4km,restaurant_count_4_5km,estimated_rent,compensation,discounted delivery,free delivery,gem,item discount,menu discount,voucher wallet,incentives_type,incentive_delivery,incentive_menu_discount,incentive_voucher_wallet,rating,n_rating,total_orders,consistency
0,7051337,1305576462,2023-10-10,2023-10-10 15:30:22,50868,"SHAGHF, Jumeirah 3",coffee,food,0.00000,1.73000,57.75000,True,10.00000,249162013.00000,55.21388,25.15585,55.23119,25.18903,15,17,27,7584090309,0.65335,840,food,697762,coffee,"Awaken Cafe, Al Wasl",55.21303,25.15146,840,4.61410,0.14160,10.25355,No,30.06471,55.21387725.155851,0.00000,0.00000,0.47137,21.00000,0.65855,105.00000,0.66518,198.00000,0.69070,296.00000,0.70457,582.00000,21.00000,84.00000,93.00000,98.00000,286.00000,90.44289,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,10.00000,voucher wallet,No,No,Yes,1.Amazing,5.None,58,0.85783
1,7051337,1313167366,2023-10-16,2023-10-16 16:43:34,50868,"SHAGHF, Jumeirah 3",coffee,food,7.50000,0.90000,26.25000,True,10.00000,250948408.00000,55.21389,25.15572,55.23119,25.18903,16,13,31,7584090309,0.65335,840,food,697762,coffee,"Awaken Cafe, Al Wasl",55.21303,25.15146,840,4.61410,0.14160,8.93051,No,30.06471,55.213885121047525.1557155663109,0.00000,0.00000,0.47137,21.00000,0.65855,105.00000,0.66603,197.00000,0.69169,295.00000,0.70585,587.00000,21.00000,84.00000,92.00000,98.00000,292.00000,90.65273,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,10.00000,voucher wallet,No,No,Yes,1.Amazing,5.None,58,0.85783


In [13]:
df_localness_filter['nearest_dist1'] = df_localness_filter['nearest_dist1']+0.001 # plus one meter to correct zero localness when both pandana_distance_realdist and nearest_dist1 are zero
df_localness_filter['pandana_distance_realdist'] = df_localness_filter['pandana_distance_realdist']+0.001
df_localness_filter['pandana_localness'] = df_localness_filter['nearest_dist1']/df_localness_filter['pandana_distance_realdist']

## Save processed data for ML model

In [16]:
%%time
df_localness_filter.to_csv('./data/food_localness_final_oct.csv',index=False)

CPU times: user 1min 21s, sys: 1.49 s, total: 1min 23s
Wall time: 1min 24s


In [52]:
# %%time
# df_localness.to_csv('./data/food_localness_final_oct(withoutfilter_rent).csv',index=False)

CPU times: user 2min 38s, sys: 1.96 s, total: 2min 40s
Wall time: 4min 49s


In [11]:
# import pandas as pd

# df_localness_filter = pd.read_csv('./data/food_localness_final_oct.csv')
# print(df_localness_filter.shape[0])

2767286
